# Introduction

TODO:
- Frame the business problem (why we care) and 3-5 potential use cases for this technology in the retail sector
- Define success in terms of 1-2 specific metrics (recommend using MAPE and RMSE)
- Provide a high-level overview of your approach
- Write a high-level overview of your data
    - Where it came from
    - How it was collected
    - Potential biases it may have
- When you're all finished, write a 1-paragraph executive summary for this engagement

# Import

TODO:
- Load pkl into a dask dataframe for parallelized processing
- Print .head() and .summary()

OPTIONAL:
- Get a free student account to AWS EC2 / GCP and distributively process your code using dask. See here: https://docs.dask.org/en/latest/setup/cloud.html

In [8]:
import pandas as pd
import dask as dd

import xgboost as xgb
import lightgbm as lgb

import seaborn as sns

# Data Exploration & Transformation

TODO:
- Check for outliers and missing data
- Check for duplicates
- Check datatypes and comment on whether they make sense
- Plot univariate / bivariate graphs to better understand your data, especially for your target variable, in seaborn
- Standardize, normalize, or log your target variable
- Split your data into training, testing, and holdout sets and explain why this step is important

# Feature Engineering

TODO:
- Discretize any categorical features
- Develop 3+ new time-related features and 3+ new product-related features as functions in a script called 'feature_engineering.py' in ./src
    - Time-related features: MonthYear, holiday flags, etc.
    - Product-related features: average number of items sold per month in each store (frequency), last time product sold in the store (recency), etc. 

# Modeling

TODO: 
- Write a function to run your lightgbm and xgboost models on your training data and output your in-sample and out-of-sample error RMSE and MAPE
- Use your function to 'tune' both algorithms by passing in different combinations of hyperparameters (max depth, subsampling, etc.) to optimize for OOS RMSE
- Plot your OOS RMSE on the y-axis and your various hyperparameter combinations on the x-axis to select the best combination
- Retrain your models on your train + test data and measure your IS and OOS error metrics on your holdout data 

OPTIONAL:
- Validate that your results are accurate using k-fold cross-validation. Note that this step is extremely important, but can be time-intensive; including it here as optional to be conscious of our time constraints.

# Interpretation

TODO:
- Create a SHAP plot for our best model and interpret its results
